In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import argparse
import numpy as np

import ray
import resnet.models as models
import random,time
from time import sleep
import copy 

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
import argparse
import numpy as np
import os
import shutil
from torch.utils.tensorboard import SummaryWriter


if ray.is_initialized():
    ray.shutdown()
ray.init(memory=10000000000,object_store_memory=6000000000)


/userhome/34/gyu/anaconda3/envs/pytorch_env/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/userhome/34/gyu/anaconda3/envs/pytorch_env/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/userhome/34/gyu/anaconda3/envs/pytorch_env/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)]

{'node_ip_address': '10.21.5.174',
 'redis_address': '10.21.5.174:14237',
 'object_store_address': '/tmp/ray/session_2019-11-05_22-11-00_371096_3747/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2019-11-05_22-11-00_371096_3747/sockets/raylet',
 'webui_url': None,
 'session_dir': '/tmp/ray/session_2019-11-05_22-11-00_371096_3747'}

In [2]:
arch = "resnet"
# depth = 56
depth = 164
cuda = torch.cuda.is_available()
cuda = False
seed = 1
save = "./logs"
dataset = "cifar10"
batch_size = 64
test_batch_size = 500
kwargs = {'num_workers': 1, 'pin_memory': True} if cuda else {}

ckpt_ssp_resnet = "./ckpt_ssp_resnet_slim/checkpoint.pth.tar"

lr = 0.01 # changed from 0.01 to 0.001 at eval acc 57%
momentum=0.9
weight_decay=5e-4 # changed from 1e-4 to 5e-4
log_interval=100
start_epoch = 0
epochs=160

# network slimming
sparsity = True
bn_lambda = 1e-5

In [3]:
if not os.path.exists(save):
    os.makedirs(save)
# if not os.path.exists(ckpt_ssp_resnet):
#     os.makedirs(ckpt_ssp_resnet)
    

In [4]:
if dataset == "cifar10":
    train_loader = torch.utils.data.DataLoader(
        datasets.CIFAR10('./data.cifar10', train=True, download=True,
                       transform=transforms.Compose([
                           transforms.Pad(4),
                           transforms.RandomCrop(32),
                           transforms.RandomHorizontalFlip(),
                           transforms.ToTensor(),
                           transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
                       ])),
        batch_size=batch_size, shuffle=True, **kwargs)
    test_loader = torch.utils.data.DataLoader(
        datasets.CIFAR10('./data.cifar10', train=False, transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
                       ])),
        batch_size=test_batch_size, shuffle=True, **kwargs)

Files already downloaded and verified


In [5]:
@ray.remote(num_gpus=0.2)
# @ray.remote
class ParameterServer():
    def __init__(self,lr,num_workers,stalness_limit,test_loader,resume_from_ckpt=None):
        self.lr = lr
        self.model = models.__dict__[arch](dataset=dataset,depth=depth)
        self.stalness_table = [0] * num_workers
        self.stalness_limit = stalness_limit 
        self.global_step = 0
        self.eva_model = models.__dict__[arch](dataset=dataset,depth=depth)
#         self.eva_model.eval()
        self.optimizer = optim.SGD(self.model.parameters(),
                          lr=lr,
                          momentum=momentum,
                          weight_decay=weight_decay)
        self.test_loader = test_loader
        if cuda:
            self.model.cuda()
            self.eva_model.cuda()
        # tensorboard logger
        self.ps_writer = SummaryWriter("ssp_resnet_runs_decay_more/ps")
        
        if resume_from_ckpt:
            self.model.load_state_dict(torch.load(resume_from_ckpt))

#     # additional subgradient descent on the sparsity-induced penalty term
#     def updateBN(self):
#         for m in self.model.modules():
#             if isinstance(m, nn.BatchNorm2d):
#                 m.weight.grad.data.add_(bn_lambda*torch.sign(m.weight.data))  # L1
                
    def apply_gradients(self, gradients, wk_idx):
        print("applying gradients from the ",wk_idx, " worker")
        for idx, p in enumerate(self.model.parameters()):
            p.data -= self.lr * gradients[idx]
        self.stalness_table[wk_idx] += 1
        self.global_step += 1
#         if sparsity:
#             self.updateBN()
        print("finished applying gradients from the ",wk_idx, " worker")
        if self.global_step % 50 == 0:
            print("global_step: ",self.global_step," and prepare evaluate")
            self.evaluate()
        
    def pull_weights(self):
        return self.model.state_dict()
    
    def check_stalness(self,wk_idx):
        min_iter = min(self.stalness_table)
        return self.stalness_table[wk_idx] - min_iter < self.stalness_limit
        
    def get_stalness(self):
        return min(self.stalness_table)
    
    def evaluate(self):
        print("going to evaluate")
        test_loss = 0.
        correct = 0.
#         print("pulled weights")
        self.eva_model.load_state_dict(copy.deepcopy(self.model.state_dict()))
#         print("loaded weights")
#         print("length of the test_loader dataset is : ",len(test_loader.dataset))
        batch = iter(test_loader)
        data,target = next(batch)
        if cuda: 
            data,target = data.cuda(),target.cuda()
        data,target = Variable(data,volatile=True),Variable(target)
        output = self.eva_model(data)
        test_loss = F.cross_entropy(output,target,size_average=True)
        pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability
        correct = pred.eq(target.data.view_as(pred)).cpu().sum()
        
        #log tensorboard
        self.ps_writer.add_scalar('Accuracy/eval', (100.0 * correct) / len(data), self.global_step)
        self.ps_writer.add_scalar('Loss/eval',test_loss , self.global_step)
        
        print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.1f}%)\n'.format(
            test_loss, 
            correct, 
            len(data),
            100. * correct / len(data)))
        
    def get_model(self):
        return self.model
    
    def get_stalness_table(self):
        return self.stalness_table

In [6]:
@ray.remote(num_gpus=0.2)
# @ray.remote
def worker_task(ps,worker_index,stale_limit, train_loader,lr,momentum,weight_decay,bn_lambda,batch_size=50):
    # Initialize the model.
    model = models.__dict__[arch](dataset=dataset,depth=depth)
    local_step = 0
    optimizer = optim.SGD(model.parameters(),
                          lr=lr,
                          momentum=momentum,
                          weight_decay=weight_decay)
    if cuda:
        model.cuda()
    wk_writer = SummaryWriter("ssp_resnet_runs_decay_more/wk_"+str(worker_index))
    for batch_idx,(data,target) in enumerate(train_loader):
        if cuda:
            data,target = data.cuda(),target.cuda()
        while(local_step - ray.get(ps.get_stalness.remote()) > stale_limit):
            print(worker_index," works too fast")
            sleep(1)
            
        # Get the current weights from the parameter server.
#         print("the ",worker_index," pulls wei from ps.")
        init_wei = ray.get(ps.pull_weights.remote())
        model.load_state_dict(init_wei)
#         print("the ",worker_index," loaded the latest wei from ps.")

        # Compute an update and push it to the parameter server.        
        data, target = Variable(data), Variable(target)
        optimizer.zero_grad()
        loss = F.cross_entropy(model(data),target)
        loss.backward()
        
        # slimming 
        for m in model.modules():
            if isinstance(m, nn.BatchNorm2d):
                m.weight.grad.data.add_(bn_lambda*torch.sign(m.weight.data))  # L1
                
        grad = [p.grad for p in model.parameters()]
        local_step += 1
        ps.apply_gradients.remote(grad,worker_index)
        optimizer.step()
        wk_writer.add_scalar("Loss/worker_train",loss,local_step)
        print("the ",worker_index," has finished its ",local_step," update")

In [7]:
num_worker = 4
stalness_table = [0] * num_worker
stalness_limit = 4

# ps = ParameterServer.remote(lr,num_worker,stalness_limit,test_loader,ckpt_ssp_resnet)
ps = ParameterServer.remote(lr,num_worker,stalness_limit,test_loader)

2019-11-05 22:11:12,246	WARNING worker.py:1779 -- Warning: The actor ParameterServer has size 30746018 when pickled. It will be stored in Redis, which could cause memory issues. This may mean that its definition uses a large array or other object.


(pid=3785) /userhome/34/gyu/anaconda3/envs/pytorch_env/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
(pid=3785)   _np_qint8 = np.dtype([("qint8", np.int8, 1)])
(pid=3785) /userhome/34/gyu/anaconda3/envs/pytorch_env/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
(pid=3785)   _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
(pid=3785) /userhome/34/gyu/anaconda3/envs/pytorch_env/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)t

In [8]:
worker_tasks = [worker_task.remote(ps,i,stalness_limit,train_loader,lr,momentum,weight_decay,bn_lambda) 
                for i in range(num_worker)]

(pid=3784) /userhome/34/gyu/anaconda3/envs/pytorch_env/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
(pid=3784)   _np_qint8 = np.dtype([("qint8", np.int8, 1)])
(pid=3784) /userhome/34/gyu/anaconda3/envs/pytorch_env/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
(pid=3784)   _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
(pid=3784) /userhome/34/gyu/anaconda3/envs/pytorch_env/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)t

(pid=3785) applying gradients from the  0  worker
(pid=3788) the  0  has finished its  1  update
(pid=3785) finished applying gradients from the  0  worker
(pid=3785) applying gradients from the  0  worker
(pid=3785) finished applying gradients from the  0  worker
(pid=3788) the  0  has finished its  2  update
(pid=3785) applying gradients from the  1  worker
(pid=3784) the  1  has finished its  1  update
(pid=3785) finished applying gradients from the  1  worker
(pid=3785) applying gradients from the  0  worker
(pid=3785) finished applying gradients from the  0  worker
(pid=3788) the  0  has finished its  3  update
(pid=3785) applying gradients from the  1  worker
(pid=3785) finished applying gradients from the  1  worker
(pid=3784) the  1  has finished its  2  update
(pid=3785) applying gradients from the  2  worker
(pid=3785) finished applying gradients from the  2  worker
(pid=3786) the  2  has finished its  1  update
(pid=3785) applying gradients from the  0  worker
(pid=3788) the

In [17]:
stalness = ray.get(ps.get_stalness_table.remote())

In [18]:
stalness

[5, 4, 3, 1]

(pid=3785) applying gradients from the  0  worker
(pid=3785) finished applying gradients from the  0  worker
(pid=3788) the  0  has finished its  6  update
(pid=3788) 0  works too fast
(pid=3788) 0  works too fast
(pid=3785) applying gradients from the  1  worker
(pid=3784) the  1  has finished its  5  update
(pid=3785) finished applying gradients from the  1  worker
(pid=3788) 0  works too fast
(pid=3785) applying gradients from the  3  worker
(pid=3785) finished applying gradients from the  3  worker
(pid=3787) the  3  has finished its  2  update


In [20]:
def save_checkpoint(state, is_best, filepath):
    torch.save(state, filepath)
    if is_best:
        shutil.copyfile(os.path.join(filepath, 'checkpoint.pth.tar'), os.path.join(filepath, 'model_best.pth.tar'))

import datetime

while True:
    wei = ray.get(ps.pull_weights.remote())
    save_checkpoint(wei,False,ckpt_ssp_resnet)
    print("saved ckpt at: ", )
    print(datetime.datetime.now())
    time.sleep(600)

saved ckpt at: 
2019-11-05 22:13:28.691633
(pid=3785) applying gradients from the  1  worker
(pid=3784) the  1  has finished its  6  update
(pid=3785) finished applying gradients from the  1  worker
(pid=3785) applying gradients from the  2  worker
(pid=3786) the  2  has finished its  5  update
(pid=3785) finished applying gradients from the  2  worker
(pid=3785) applying gradients from the  0  worker
(pid=3788) the  0  has finished its  7  update
(pid=3785) finished applying gradients from the  0  worker
(pid=3788) 0  works too fast
(pid=3788) 0  works too fast
(pid=3785) applying gradients from the  3  worker
(pid=3785) finished applying gradients from the  3  worker
(pid=3787) the  3  has finished its  3  update
(pid=3785) applying gradients from the  1  worker
(pid=3784) the  1  has finished its  7  update
(pid=3785) finished applying gradients from the  1  worker
(pid=3785) applying gradients from the  2  worker
(pid=3786) the  2  has finished its  6  update
(pid=3785) finished ap

(pid=3785) applying gradients from the  3  worker
(pid=3785) finished applying gradients from the  3  worker
(pid=3785) applying gradients from the  2  worker
(pid=3787) the  3  has finished its  12  update
(pid=3786) the  2  has finished its  14  update
(pid=3785) finished applying gradients from the  2  worker
(pid=3788) the  0  has finished its  16  update
(pid=3785) applying gradients from the  0  worker
(pid=3785) finished applying gradients from the  0  worker
(pid=3785) applying gradients from the  1  worker
(pid=3785) finished applying gradients from the  1  worker
(pid=3784) the  1  has finished its  16  update
(pid=3785) applying gradients from the  0  worker
(pid=3785) finished applying gradients from the  0  worker
(pid=3788) the  0  has finished its  17  update
(pid=3788) 0  works too fast
(pid=3785) applying gradients from the  3  worker
(pid=3785) finished applying gradients from the  3  worker
(pid=3787) the  3  has finished its  13  update
(pid=3785) applying gradients

(pid=3785) finished applying gradients from the  3  worker
(pid=3787) the  3  has finished its  22  update
(pid=3785) applying gradients from the  0  worker
(pid=3788) the  0  has finished its  26  update
(pid=3785) finished applying gradients from the  0  worker
(pid=3785) applying gradients from the  1  worker
(pid=3784) the  1  has finished its  26  update
(pid=3785) finished applying gradients from the  1  worker
(pid=3785) applying gradients from the  2  worker
(pid=3785) finished applying gradients from the  2  worker
(pid=3786) the  2  has finished its  25  update
(pid=3785) applying gradients from the  3  worker
(pid=3785) finished applying gradients from the  3  worker
(pid=3785) global_step:  100  and prepare evaluate
(pid=3785) going to evaluate
(pid=3787) the  3  has finished its  23  update
(pid=3788) the  0  has finished its  27  update
(pid=3784) the  1  has finished its  27  update
(pid=3786) the  2  has finished its  26  update
(pid=3785) 
(pid=3785) Test set: Average 

(pid=3785) finished applying gradients from the  3  worker
(pid=3787) the  3  has finished its  32  update
(pid=3785) applying gradients from the  2  worker
(pid=3785) finished applying gradients from the  2  worker
(pid=3786) the  2  has finished its  35  update
(pid=3785) applying gradients from the  1  worker
(pid=3784) the  1  has finished its  36  update
(pid=3785) finished applying gradients from the  1  worker
(pid=3785) applying gradients from the  0  worker
(pid=3788) the  0  has finished its  36  update
(pid=3785) finished applying gradients from the  0  worker
(pid=3785) applying gradients from the  3  worker
(pid=3785) finished applying gradients from the  3  worker
(pid=3787) the  3  has finished its  33  update
(pid=3785) applying gradients from the  2  worker
(pid=3785) finished applying gradients from the  2  worker
(pid=3786) the  2  has finished its  36  update
(pid=3785) applying gradients from the  1  worker
(pid=3784) the  1  has finished its  37  update
(pid=3785)

2019-11-05 22:24:31,353	WARNING worker.py:1779 -- The log monitor on node gpu-comp-204 failed with the following error:
Traceback (most recent call last):
  File "/userhome/34/gyu/anaconda3/envs/pytorch_env/lib/python3.7/site-packages/ray/log_monitor.py", line 202, in check_log_files_and_publish_updates
    next_line = file_info.file_handle.readline()
OSError: [Errno 12] Cannot allocate memory

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/userhome/34/gyu/anaconda3/envs/pytorch_env/lib/python3.7/site-packages/ray/log_monitor.py", line 301, in <module>
    log_monitor.run()
  File "/userhome/34/gyu/anaconda3/envs/pytorch_env/lib/python3.7/site-packages/ray/log_monitor.py", line 252, in run
    anything_published = self.check_log_files_and_publish_updates()
  File "/userhome/34/gyu/anaconda3/envs/pytorch_env/lib/python3.7/site-packages/ray/log_monitor.py", line 215, in check_log_files_and_publish_updates
    file_info.ful

RayActorError: The actor died unexpectedly before finishing this task.